# BTree

## Structure

In [149]:
data Btree a = Leaf a | Node (Btree a) (Btree a)

bt0 = Node (Leaf 3) (Node (Leaf 3) (Leaf 4))
bt1 = Node (Leaf 3) (Node (Leaf 3) (Leaf 4))
bt2 = Node (Leaf 4) (Node (Leaf 3) (Leaf 4))
bt3 = Node (Leaf 4) (Node (Leaf 3) (Leaf 5))

In [150]:
instance Show a => Show (Btree a) where
    show :: Btree a -> String
    show (Leaf a) = "Leaf " ++ show a
    show (Node l r) = "Node (" ++ show l ++ ") (" ++ show r ++ ")"

In [151]:
instance Eq a => Eq (Btree a) where
    (==) :: Btree a -> Btree a -> Bool
    (Leaf a) == (Leaf b) = a == b
    (Node l1 r1) == (Node l2 r2) = l1 == l2 && r1 == r2
    _ == _ = False

bt0 == bt1
bt1 == bt2

True

False

In [152]:
instance Ord a => Ord (Btree a) where
    compare :: Btree a -> Btree a -> Ordering
    (Leaf a) `compare` (Leaf b) = a `compare` b
    (Node l1 r1) `compare` (Node l2 r2) = case l1 `compare` l2 of
                                                EQ -> r1 `compare` r2
                                                o -> o

bt1 < bt2
bt3 > bt2


True

True

In [153]:
foldbt :: (a -> b) -> (b -> b -> b) -> Btree a -> b
foldbt fl _ (Leaf a) = fl a
foldbt fl fn (Node r l) = fn (foldbt fl fn l) (foldbt fl fn r)

foldbt id (\a b -> a + b) bt1

Line 5: Avoid lambda
Found:
\ a b -> a + b
Why not:
(+)

10

In [154]:
mapbt :: (a -> b) -> Btree a -> Btree b
mapbt f (Leaf a) = Leaf $ f a
mapbt f (Node l r) = Node (mapbt f l) (mapbt f r)

mapbt (+1) bt1

Node (Leaf 4) (Node (Leaf 4) (Leaf 5))

## Functor

In [155]:
instance Functor Btree where
    fmap :: (a -> b) -> Btree a -> Btree b
    fmap f (Leaf a) = Leaf $ f a
    fmap f (Node l r) = Node (fmap f l) (fmap f l)

instance Applicative Btree where
    pure :: a -> Btree a
    pure = Leaf

    (<*>) :: Btree (a -> b) -> Btree a -> Btree b
    Leaf f <*> Leaf x = Leaf $ f x
    f <*> Node x y = Node (f <*> x) (f <*> y)
    -- Leaf f <*> Node x y = Node (f `fmap` x) (f `fmap` y)
    _ <*> _ = error "must provide Leaf with function"

instance Monad Btree where
    return = pure
    
    (>>=) :: Btree a -> (a -> Btree b) -> Btree b
    Leaf a >>= f = f a
    Node l r >>= f = Node (l >>= \ll -> f ll) (r >>= \rr -> f rr)


bt0 >>= (\x -> Node (Leaf x) (Leaf (x + 1)))

Leaf (+2) <*> Node (Node (Leaf 1) (Leaf 3)) (Leaf 2)
-- Node (Leaf (+1)) (Leaf (2+)) <*> Node (Leaf 1) (Leaf 2)

Node (Node (Leaf 3) (Leaf 4)) (Node (Node (Leaf 3) (Leaf 4)) (Node (Leaf 4) (Leaf 5)))

Node (Node (Leaf 3) (Leaf 5)) (Leaf 4)

# KTree

In [156]:
data Ktree a = KLeaf a | KNode (Ktree a) a (Ktree a)

kt0 = KNode (KLeaf 3) 3 (KNode (KLeaf 3) 4 (KLeaf 4))
kt1 = KNode (KLeaf 3) 3 (KNode (KLeaf 3) 4 (KLeaf 4))
kt2 = KNode (KLeaf 3) 4 (KNode (KLeaf 3) 4 (KLeaf 4))
kt3 = KNode (KLeaf 2) 3 (KNode (KLeaf 3) 4 (KLeaf 4))

In [157]:
instance Show a => Show (Ktree a) where
    show :: Ktree a -> String
    show (KLeaf a) = "Leaf " ++ show a
    show (KNode l c r) = "Node (" ++ show l ++ ") " ++ show c ++ " (" ++ show r ++ ")"

kt1

Node (Leaf 3) 3 (Node (Leaf 3) 4 (Leaf 4))

In [158]:
instance Eq a => Eq (Ktree a) where
    (KLeaf a) == (KLeaf b) = a == b
    (KNode l1 c1 r1) == (KNode l2 c2 r2) = c1 == c2 && l1 == l2 && r1 == r2
    _ == _ = False

kt0 == kt1
kt1 == kt2

True

False

In [159]:
instance Ord a => Ord (Ktree a) where
    (KLeaf a) `compare` (KLeaf b) = a `compare` b
    (KNode l1 c1 r1) `compare` (KNode l2 c2 r2) = case c1 `compare`c2 of 
                                    EQ -> case l1 `compare` l2 of 
                                        EQ -> r1 `compare` r2
                                        o -> o
                                    o -> o



kt0 > kt1
kt2 > kt1
kt3 < kt2

False

True

True

In [160]:
foldkt :: (a -> b) -> (b -> a -> b -> b) -> Ktree a -> b
foldkt fl _ (KLeaf a) = fl a
foldkt fl fn (KNode l a r) = fn (foldkt fl fn l) a (foldkt fl fn r)

foldkt id (\l a r -> l + a + r ) kt0

17

In [161]:
mapkt :: (a -> b) -> Ktree a -> Ktree b
mapkt f (KLeaf a) = KLeaf $ f a
mapkt f (KNode l c r) = KNode (mapkt f l) (f c) (mapkt f r)

mapkt show kt0

Node (Leaf "3") "3" (Node (Leaf "3") "4" (Leaf "4"))

## Functor

In [162]:
instance Functor Ktree where
    fmap :: (a -> b) -> Ktree a -> Ktree b
    fmap f (KLeaf x) = KLeaf $ f x
    fmap f (KNode l c r) = KNode (fmap f l) (f c) (fmap f r)

instance Applicative Ktree where
    pure :: a -> Ktree a
    pure = KLeaf

    (<*>) :: Ktree (a -> b) -> Ktree a -> Ktree b
    KLeaf f <*> KLeaf x = KLeaf $ f x
    KLeaf f <*> KNode l c r = KNode (f <$> l) (f c) (f <$> r)
    _ <*> _ = error "must provide Leaf with function"

instance Monad Ktree where
    return = pure
    
    (>>=) :: Ktree a -> (a -> Ktree b) -> Ktree b
    KLeaf a >>= f = f a
    KNode l c r >>= f = KNode (l >>= \ll -> f ll) m (r >>= \rr -> f rr) where (KLeaf m) = f c

# RTree

In [163]:
data Rtree a = Rtree a [Rtree a]

rt0 = Rtree 1 [Rtree 21 [], Rtree 22 [], Rtree 23 [Rtree 51 [], Rtree 52 []], Rtree 24 []]

In [164]:
instance Show a => Show (Rtree a) where
    show :: Rtree a -> String
    show (Rtree a xs) = case length xs of
                0 -> "Rtree " ++ show a ++ " [], "
                other -> "Rtree " ++ show a ++ " [" ++ concatMap show xs ++ "] "

rt0

Rtree 1 [Rtree 21 [], Rtree 22 [], Rtree 23 [Rtree 51 [], Rtree 52 [], ] Rtree 24 [], ]

In [165]:
instance Eq a => Eq (Rtree a) where
    (==) :: Rtree a -> Rtree a -> Bool
    Rtree a as == Rtree b bs = a == b && as == bs

In [166]:
instance Ord a => Ord (Rtree a) where
    compare :: Rtree a -> Rtree a -> Ordering
    Rtree a as `compare` Rtree b bs = case a `compare` b of
                            EQ -> EQ
                            other -> as `compare` bs

In [167]:
foldrt :: (a -> [b] -> b) -> Rtree a -> b
foldrt f (Rtree a []) = f a []
foldrt f (Rtree a xs) = f a (map (\x -> foldrt f x) xs)

foldrt (\a xs -> show a ++ show xs) rt0
foldrt (\a xs -> a + sum(xs)) rt0

Line 3: Avoid lambda
Found:
\ x -> foldrt f x
Why not:
foldrt fLine 6: Redundant bracket
Found:
(xs)
Why not:
xs

"1[\"21[]\",\"22[]\",\"23[\\\"51[]\\\",\\\"52[]\\\"]\",\"24[]\"]"

194

## Functor

In [169]:
instance Functor Rtree where
    fmap :: (a -> b) -> Rtree a -> Rtree b
    fmap f (Rtree a []) = Rtree (f a) []
    fmap f (Rtree a as) = Rtree (f a) (fmap (fmap f) as)

instance Applicative Rtree where
    pure :: a -> Rtree a
    pure a = Rtree a []

    (<*>) :: Rtree (a -> b) -> Rtree a -> Rtree b
    Rtree f fs <*> Rtree a as = Rtree (f a) [g <*> b | g <- fs, b <- as]

instance Monad Rtree where
    return = pure

    (>>=) :: Rtree a -> (a -> Rtree b) -> Rtree b
    Rtree a as >>= f = Rtree m [f b | Rtree b bs <- as] where Rtree m [] = f a

[(+1), (+2)] <*> [2, 3]

Rtree (+1) [Rtree (+1) [], Rtree (+2) []] <*> Rtree 0 [Rtree 2 [], Rtree 3 []]

[[1, 2], [3,4]] >>= (\x -> [show x])

-- Rtree (0 ::Int) [Rtree 2 [], Rtree 3 []] >>= (\(Rtree a as) -> Rtree (show a) [pure (show b) | b <- as])


[3,4,4,5]

Rtree 1 [Rtree 3 [], Rtree 4 [], Rtree 4 [], Rtree 5 [], ]

["[1,2]","[3,4]"]

In [170]:
fmap (+1) (Just 5)

Just 6

In [171]:
Just (+1) <*> Just 5

Just 6

# More functions

In [225]:
data Etree a = Empty | ENode (Etree a) a (Etree a) deriving Show

numberTree :: Etree a -> Etree (Int, a)
numberTree t = nt t 0 where
    nt :: Etree a -> Int -> Etree (Int, a)
    nt (ENode Empty c Empty) n = ENode Empty (n,c) Empty
    -- nt (ENode x c y) n = (ENode (nt x (n+1)) (n,c) (nt y (n+1)))
    nt (ENode x c y) n = let 
                        lt = nt x n
                        ml = maxTree lt n
                        lr = nt y ml
                        mr = maxTree lr ml
                        in
                        (ENode lt (mr + 1, c) lr)


maxTree :: Etree (Int, a) -> Int -> Int
maxTree Empty n = n
maxTree (ENode r (nc,c) l) n = max (max (maxTree r n) (maxTree l n)) nc

a = ENode (ENode Empty 2 Empty) 3 (ENode Empty 6 Empty)
b = ENode (ENode Empty (6,3) Empty) (2,3) (ENode Empty (10,1) Empty)

-- numberTree a
maxTree b 0

Line 14: Redundant bracket
Found:
let
  lt = nt x n
  ml = maxTree lt n
  lr = nt y ml
  mr = maxTree lr ml
in (ENode lt (mr + 1, c) lr)
Why not:
let
  lt = nt x n
  ml = maxTree lt n
  lr = nt y ml
  mr = maxTree lr ml
in ENode lt (mr + 1, c) lr

10